# Antrenarea RL pentru echilibrarea Cartpole

Acest notebook face parte din [Curriculumul AI pentru Începători](http://aka.ms/ai-beginners). A fost inspirat de [tutorialul oficial PyTorch](https://pytorch.org/tutorials/intermediate/reinforcement_q_learning.html) și de [această implementare Cartpole în PyTorch](https://github.com/yc930401/Actor-Critic-pytorch).

În acest exemplu, vom folosi RL pentru a antrena un model să echilibreze un stâlp pe un cărucior care se poate deplasa la stânga și la dreapta pe o scară orizontală. Vom folosi mediul [OpenAI Gym](https://www.gymlibrary.ml/) pentru a simula stâlpul.

> **Note**: Puteți rula codul lecției local (de exemplu, din Visual Studio Code), caz în care simularea se va deschide într-o fereastră nouă. Când rulați codul online, este posibil să fie nevoie să faceți unele ajustări ale codului, așa cum este descris [aici](https://towardsdatascience.com/rendering-openai-gym-envs-on-binder-and-google-colab-536f99391cc7).

Vom începe prin a ne asigura că Gym este instalat:


In [ ]:
import sys
!{sys.executable} -m pip install gym

Acum să creăm mediul CartPole și să vedem cum să lucrăm cu el. Un mediu are următoarele proprietăți:

* **Spațiul de acțiuni** este setul de acțiuni posibile pe care le putem efectua la fiecare pas al simulării
* **Spațiul de observații** este spațiul observațiilor pe care le putem face


In [ ]:
import gym

env = gym.make("CartPole-v1")

print(f"Action space: {env.action_space}")
print(f"Observation space: {env.observation_space}")

Să vedem cum funcționează simularea. Următorul buclă rulează simularea până când `env.step` nu mai returnează indicatorul de terminare `done`. Vom alege acțiuni în mod aleatoriu folosind `env.action_space.sample()`, ceea ce înseamnă că experimentul va eșua probabil foarte repede (mediul CartPole se termină atunci când viteza CartPole, poziția sau unghiul său depășesc anumite limite).

> Simularea se va deschide într-o fereastră nouă. Poți rula codul de mai multe ori și observa cum se comportă.


In [ ]:
env.reset()

done = False
total_reward = 0
while not done:
   env.render()
   obs, rew, done, info = env.step(env.action_space.sample())
   total_reward += rew
   print(f"{obs} -> {rew}")
print(f"Total reward: {total_reward}")

Poți observa că observațiile conțin 4 numere. Acestea sunt:
- Poziția căruciorului
- Viteza căruciorului
- Unghiul stâlpului
- Rata de rotație a stâlpului

`rew` este recompensa pe care o primim la fiecare pas. Poți observa că, în mediul CartPole, primești 1 punct pentru fiecare pas de simulare, iar scopul este să maximizezi recompensa totală, adică timpul în care CartPole reușește să se echilibreze fără să cadă.

În timpul învățării prin întărire, scopul nostru este să antrenăm o **politică** $\pi$, care pentru fiecare stare $s$ ne va spune ce acțiune $a$ să luăm, practic $a = \pi(s)$.

Dacă dorești o soluție probabilistică, poți considera politica ca returnând un set de probabilități pentru fiecare acțiune, adică $\pi(a|s)$ ar însemna probabilitatea că ar trebui să luăm acțiunea $a$ în starea $s$.

## Metoda Gradientului Politicii

În cel mai simplu algoritm RL, numit **Gradientul Politicii**, vom antrena o rețea neuronală pentru a prezice următoarea acțiune.


In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import torch

num_inputs = 4
num_actions = 2

model = torch.nn.Sequential(
    torch.nn.Linear(num_inputs, 128, bias=False, dtype=torch.float32),
    torch.nn.ReLU(),
    torch.nn.Linear(128, num_actions, bias = False, dtype=torch.float32),
    torch.nn.Softmax(dim=1)
)

Vom antrena rețeaua rulând multe experimente și actualizând rețeaua noastră după fiecare rulare. Să definim o funcție care va rula experimentul și va returna rezultatele (așa-numitul **trace**) - toate stările, acțiunile (și probabilitățile recomandate ale acestora) și recompensele:


In [ ]:
def run_episode(max_steps_per_episode = 10000,render=False):    
    states, actions, probs, rewards = [],[],[],[]
    state = env.reset()
    for _ in range(max_steps_per_episode):
        if render:
            env.render()
        action_probs = model(torch.from_numpy(np.expand_dims(state,0)))[0]
        action = np.random.choice(num_actions, p=np.squeeze(action_probs.detach().numpy()))
        nstate, reward, done, info = env.step(action)
        if done:
            break
        states.append(state)
        actions.append(action)
        probs.append(action_probs.detach().numpy())
        rewards.append(reward)
        state = nstate
    return np.vstack(states), np.vstack(actions), np.vstack(probs), np.vstack(rewards)

Poți rula un episod cu rețeaua neantrenată și observa că recompensa totală (cunoscută și ca lungimea episodului) este foarte mică:


In [ ]:
s, a, p, r = run_episode()
print(f"Total reward: {np.sum(r)}")

Unul dintre aspectele dificile ale algoritmului de gradient de politică este utilizarea **recompenselor reduse**. Ideea este că calculăm vectorul recompenselor totale la fiecare pas al jocului, iar în acest proces reducem recompensele timpurii folosind un coeficient $gamma$. De asemenea, normalizăm vectorul rezultat, deoarece îl vom folosi ca greutate pentru a influența antrenamentul nostru:


In [ ]:
eps = 0.0001

def discounted_rewards(rewards,gamma=0.99,normalize=True):
    ret = []
    s = 0
    for r in rewards[::-1]:
        s = r + gamma * s
        ret.insert(0, s)
    if normalize:
        ret = (ret-np.mean(ret))/(np.std(ret)+eps)
    return ret

Acum să trecem la antrenamentul propriu-zis! Vom rula 300 de episoade, iar la fiecare episod vom face următoarele:

1. Rulăm experimentul și colectăm traseul
1. Calculăm diferența (`gradients`) între acțiunile întreprinse și probabilitățile prezise. Cu cât diferența este mai mică, cu atât suntem mai siguri că am luat acțiunea corectă.
1. Calculăm recompensele actualizate și înmulțim gradientele cu recompensele actualizate - acest lucru va asigura că pașii cu recompense mai mari vor avea un impact mai mare asupra rezultatului final decât cei cu recompense mai mici.
1. Acțiunile țintă așteptate pentru rețeaua noastră neuronală vor fi parțial preluate din probabilitățile prezise în timpul rulării și parțial din gradienții calculați. Vom folosi parametrul `alpha` pentru a determina în ce măsură sunt luate în considerare gradientele și recompensele - acest lucru se numește *rata de învățare* a algoritmului de întărire.
1. În cele din urmă, antrenăm rețeaua noastră pe stări și acțiuni așteptate și repetăm procesul.


In [ ]:
optimizer = torch.optim.Adam(model.parameters(), lr=0.01)

def train_on_batch(x, y):
    x = torch.from_numpy(x)
    y = torch.from_numpy(y)
    optimizer.zero_grad()
    predictions = model(x)
    loss = -torch.mean(torch.log(predictions) * y)
    loss.backward()
    optimizer.step()
    return loss

In [ ]:
alpha = 1e-4

history = []
for epoch in range(300):
    states, actions, probs, rewards = run_episode()
    one_hot_actions = np.eye(2)[actions.T][0]
    gradients = one_hot_actions-probs
    dr = discounted_rewards(rewards)
    gradients *= dr
    target = alpha*np.vstack([gradients])+probs
    train_on_batch(states,target)
    history.append(np.sum(rewards))
    if epoch%100==0:
        print(f"{epoch} -> {np.sum(rewards)}")

plt.plot(history)

Acum să rulăm episodul cu redare pentru a vedea rezultatul:


In [ ]:
_ = run_episode(render=True)

Sperăm că acum poți observa că bara se poate echilibra destul de bine!

## Modelul Actor-Critic

Modelul Actor-Critic reprezintă o dezvoltare ulterioară a gradientelor de politică, în care construim o rețea neuronală pentru a învăța atât politica, cât și recompensele estimate. Rețeaua va avea două ieșiri (sau poate fi privită ca două rețele separate):
* **Actorul** va recomanda acțiunea de luat, oferindu-ne distribuția probabilistică a stării, la fel ca în modelul cu gradient de politică.
* **Criticul** va estima ce recompensă ar rezulta din acele acțiuni. Acesta returnează recompensele totale estimate în viitor pentru starea dată.

Să definim un astfel de model:


In [ ]:
from itertools import count
import torch.nn.functional as F

In [ ]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
env = gym.make("CartPole-v1")

state_size = env.observation_space.shape[0]
action_size = env.action_space.n
lr = 0.0001

class Actor(torch.nn.Module):
    def __init__(self, state_size, action_size):
        super(Actor, self).__init__()
        self.state_size = state_size
        self.action_size = action_size
        self.linear1 = torch.nn.Linear(self.state_size, 128)
        self.linear2 = torch.nn.Linear(128, 256)
        self.linear3 = torch.nn.Linear(256, self.action_size)

    def forward(self, state):
        output = F.relu(self.linear1(state))
        output = F.relu(self.linear2(output))
        output = self.linear3(output)
        distribution = torch.distributions.Categorical(F.softmax(output, dim=-1))
        return distribution


class Critic(torch.nn.Module):
    def __init__(self, state_size, action_size):
        super(Critic, self).__init__()
        self.state_size = state_size
        self.action_size = action_size
        self.linear1 = torch.nn.Linear(self.state_size, 128)
        self.linear2 = torch.nn.Linear(128, 256)
        self.linear3 = torch.nn.Linear(256, 1)

    def forward(self, state):
        output = F.relu(self.linear1(state))
        output = F.relu(self.linear2(output))
        value = self.linear3(output)
        return value

Ar trebui să modificăm ușor funcțiile noastre `discounted_rewards` și `run_episode`:


In [ ]:
def discounted_rewards(next_value, rewards, masks, gamma=0.99):
    R = next_value
    returns = []
    for step in reversed(range(len(rewards))):
        R = rewards[step] + gamma * R * masks[step]
        returns.insert(0, R)
    return returns

def run_episode(actor, critic, n_iters):
    optimizerA = torch.optim.Adam(actor.parameters())
    optimizerC = torch.optim.Adam(critic.parameters())
    for iter in range(n_iters):
        state = env.reset()
        log_probs = []
        values = []
        rewards = []
        masks = []
        entropy = 0
        env.reset()

        for i in count():
            env.render()
            state = torch.FloatTensor(state).to(device)
            dist, value = actor(state), critic(state)

            action = dist.sample()
            next_state, reward, done, _ = env.step(action.cpu().numpy())

            log_prob = dist.log_prob(action).unsqueeze(0)
            entropy += dist.entropy().mean()

            log_probs.append(log_prob)
            values.append(value)
            rewards.append(torch.tensor([reward], dtype=torch.float, device=device))
            masks.append(torch.tensor([1-done], dtype=torch.float, device=device))

            state = next_state

            if done:
                print('Iteration: {}, Score: {}'.format(iter, i))
                break


        next_state = torch.FloatTensor(next_state).to(device)
        next_value = critic(next_state)
        returns = discounted_rewards(next_value, rewards, masks)

        log_probs = torch.cat(log_probs)
        returns = torch.cat(returns).detach()
        values = torch.cat(values)

        advantage = returns - values

        actor_loss = -(log_probs * advantage.detach()).mean()
        critic_loss = advantage.pow(2).mean()

        optimizerA.zero_grad()
        optimizerC.zero_grad()
        actor_loss.backward()
        critic_loss.backward()
        optimizerA.step()
        optimizerC.step()


Acum vom rula bucla principală de antrenament. Vom folosi procesul manual de antrenare a rețelei prin calcularea funcțiilor de pierdere corespunzătoare și actualizarea parametrilor rețelei:


In [ ]:

actor = Actor(state_size, action_size).to(device)
critic = Critic(state_size, action_size).to(device)
run_episode(actor, critic, n_iters=100)

In [ ]:
env.close()

## Concluzii

Am văzut două algoritmi RL în această demonstrație: gradient de politică simplu și actor-critic mai sofisticat. Puteți observa că acești algoritmi operează cu noțiuni abstracte de stare, acțiune și recompensă - astfel, pot fi aplicați în medii foarte diferite.

Învățarea prin întărire ne permite să descoperim cea mai bună strategie pentru a rezolva problema doar analizând recompensa finală. Faptul că nu avem nevoie de seturi de date etichetate ne permite să repetăm simulările de mai multe ori pentru a optimiza modelele noastre. Totuși, există încă multe provocări în RL, pe care le puteți descoperi dacă decideți să vă concentrați mai mult pe acest domeniu fascinant al inteligenței artificiale.



---

**Declinare de responsabilitate**:  
Acest document a fost tradus folosind serviciul de traducere AI [Co-op Translator](https://github.com/Azure/co-op-translator). Deși ne străduim să asigurăm acuratețea, vă rugăm să rețineți că traducerile automate pot conține erori sau inexactități. Documentul original în limba sa natală ar trebui considerat sursa autoritară. Pentru informații critice, se recomandă traducerea profesională realizată de un specialist uman. Nu ne asumăm responsabilitatea pentru eventualele neînțelegeri sau interpretări greșite care pot apărea din utilizarea acestei traduceri.
